In [6]:
import sys, pathlib
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
from src import config as cfg
from src.rebalanceo_v2 import resolver_optimizacion_v2, elegir_w_star_v2

PROJECT_ROOT = pathlib.Path().resolve().parent.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# === Paths dinámicos ===
if cfg.MODEL_TYPE == "lstm":
    from tensorflow import keras
    MODEL_PATH = cfg.MODELS / cfg.LSTM_MODEL_NAME
    DATA_PATH = cfg.DATA / "processed" / "lstm_data.pkl"
    model = keras.models.load_model(MODEL_PATH, compile=False)

elif cfg.MODEL_TYPE == "lstm5d":
    from tensorflow import keras
    MODEL_PATH = cfg.MODELS / cfg.LSTM5D_MODEL_NAME
    DATA_PATH = cfg.DATA / "processed" / "lstm5d_data.pkl"
    model = keras.models.load_model(MODEL_PATH, compile=False)
    scaler_lstm5d = joblib.load(cfg.MODELS / "scaler_X_lstm5d.pkl")

elif cfg.MODEL_TYPE == "gru5d":
    from tensorflow import keras
    MODEL_PATH = cfg.MODELS / cfg.GRU5D_MODEL_NAME
    DATA_PATH = cfg.DATA / "processed" / "gru5d_data.pkl"
    model = keras.models.load_model(MODEL_PATH, compile=False)

elif cfg.MODEL_TYPE == "xgb":
    MODEL_PATH = cfg.MODELS / cfg.XGB_MODEL_NAME
    DATA_PATH = cfg.DATA / "processed" / "xgb_data.pkl"
    model = joblib.load(MODEL_PATH)

else:
    raise ValueError(f"Modelo '{cfg.MODEL_TYPE}' no soportado")

PRICES_PATH = cfg.DATA / "raw" / "prices.parquet"
print(f"🧠 Modelo activo: {cfg.MODEL_TYPE}")

# === Cargar datos ===
df_prices = pd.read_parquet(PRICES_PATH).sort_index()
data = joblib.load(DATA_PATH)
tickers = data["tickers"]
df_prices = df_prices[tickers]
df_ret = np.log(df_prices / df_prices.shift(1)).dropna()
ret5 = df_ret.rolling(5).sum()
vol5 = df_ret.rolling(5).std()
momentum = (ret5 / (vol5 + 1e-6)).shift(1)
df_feat = pd.concat([df_ret.shift(1), momentum], axis=1).dropna()
print("✅ df_feat shape:", df_feat.shape)

# === Rebalanceo con Opt-V2 ===
def rebalancear_en_fecha(fecha, df_feat, model, w_prev):
    try:
        idx = df_feat.index.get_loc(fecha)
        ventana = df_feat.iloc[idx - cfg.WINDOW: idx]

        if cfg.MODEL_TYPE == "lstm":
            scaler = StandardScaler()
            X_input = scaler.fit_transform(ventana.values)
            X_input = np.expand_dims(X_input, 0)
            r_hat = model.predict(X_input, verbose=0)[0]

        elif cfg.MODEL_TYPE == "lstm5d":
            X_input = scaler_lstm5d.transform(ventana.values).reshape(1, cfg.WINDOW, -1)
            r_hat = model.predict(X_input, verbose=0)[0]

        elif cfg.MODEL_TYPE == "gru5d":
            ventana_ret = df_ret.iloc[idx - cfg.WINDOW: idx]
            ret5 = df_ret.rolling(5).sum()
            vol5 = df_ret.rolling(5).std()
            momentum = (ret5 / (vol5 + 1e-6)).shift(1)
            ventana_mom = momentum.loc[ventana_ret.index]
            ventana_full = pd.concat([ventana_ret, ventana_mom], axis=1)
            scaler = StandardScaler()
            X_input = scaler.fit_transform(ventana_full.values).reshape(1, cfg.WINDOW, -1)
            r_hat = model.predict(X_input, verbose=0)[0]

        elif cfg.MODEL_TYPE == "xgb":
            scaler = StandardScaler()
            X_input = scaler.fit_transform(ventana.values)
            r_hat = np.array([
                model[i].predict(X_input[-1].reshape(1, -1))[0]
                for i in range(X_input.shape[1])
            ])
        else:
            raise ValueError("Modelo no reconocido")

        fecha_ret = df_feat.index[idx]
        ventana_ret = df_ret.loc[fecha_ret - pd.Timedelta(days=cfg.WINDOW * 2): fecha_ret]
        Sigma = ventana_ret[-cfg.WINDOW:].cov().values

        if r_hat.shape[0] != Sigma.shape[0]:
            print(f"⚠️ Dim mismatch {fecha.date()}")
            return None

        res = resolver_optimizacion_v2(r_hat, Sigma, w_prev=w_prev, tau=0.4)
        w_star = elegir_w_star_v2(res, r_hat, Sigma)
        future = df_ret.iloc[idx: idx + cfg.REBAL_FREQ].values @ w_star
        return {"fecha": fecha, "retorno": future.sum(), "w_star": w_star}

    except Exception as e:
        print(f"ERROR {fecha.date()}: {e}")
        return None

# === Ejecutar backtest ===
fechas = df_feat.loc[cfg.START_BACKTEST:].index
resultados = []
w_prev = np.zeros(len(tickers))

for i in range(cfg.WINDOW, len(fechas) - cfg.REBAL_FREQ, cfg.REBAL_FREQ):
    fecha = fechas[i]
    out = rebalancear_en_fecha(fecha, df_feat, model, w_prev)
    if out is not None:
        resultados.append(out)
        w_prev = out["w_star"]
        print(f"✅ {fecha.date()} | Retorno {out['retorno']:.4%}")
    else:
        print(f"⚠️ {fecha.date()} | Resultado nulo")

# === Guardar resultados ===
res_df = pd.DataFrame(resultados).set_index("fecha")
joblib.dump(res_df, cfg.RESULT / f"backtest_{cfg.MODEL_TYPE}_optv2.pkl")
print("✅ Backtest guardado:", cfg.RESULT / f"backtest_{cfg.MODEL_TYPE}_optv2.pkl")


ModuleNotFoundError: No module named 'src'